In [ ]:
import yfinance as yf

# بازه زمانی مورد نظر
start_date = '2022-01-01'
end_date = '2022-07-27'

# نماد شرکت
symbol = 'AMZN'

# دریافت داده‌های تاریخچه بازار برای نماد شرکت در بازه زمانی مشخص شده
data = yf.download(symbol, start=start_date, end=end_date)

# تعداد سهام و موجودی حساب اولیه
shares = 0
balance = 10000

# تقسیم بازه زمانی به دو قسمت
split = len(data) // 2

# خرید سهام در ابتدای بازه زمانی
buy_shares = int(balance / data['Close'][0])
buy_amount = buy_shares * data['Close'][0]
shares += buy_shares
balance -= buy_amount

# پردازش داده‌های تاریخچه بازار
for i in range(1, len(data)-1):
    # محاسبه متوسط قیمت در دو قسمت بازه زمانی
    avg_price1 = data['Close'][:i+1].mean()
    avg_price2 = data['Close'][i+1:].mean()

    # اگر متوسط قیمت در قسمت دوم بیشتر باشد، سهام فروخته و سهام جدید خریداری می‌شود
    if avg_price2 > avg_price1:
        if shares == 0:
            buy_shares = int(balance / data['Close'][i+1])
            buy_amount = buy_shares * data['Close'][i+1]
            shares += buy_shares
            balance -= buy_amount
            print("Bought", buy_shares, "shares at price", data['Close'][i+1], "for a total of", buy_amount)
        else:
            # محاسبه تعداد سهامی که برای خرید لازم است
            buy_shares = int(10000 / data['Close'][i+1+split])

            # محاسبه مبلغ پرداختی برای خرید سهام جدید
            buy_amount = buy_shares * data['Close'][i+1+split]

            # محاسبه مبلغ دریافتی برای فروش سهام قبلی
            sell_amount = shares * data['Close'][i+1+split]

            # محاسبه مبلغ پرداختی برای خرید سهام جدید پس از کسر مبلغ دریافتی از فروش سهام قبلی
            net_buy_amount = buy_amount - sell_amount

            # محاسبه نرخ سود یا ضرر برای خرید سهام جدید
            profit_rate = (data['Close'][i+1+split] - data['Close'][i+split]) / data['Close'][i+split]

            # اگر مبلغ پرداختی برای خرید سهام جدید از مقدار موجودی حساب بیشتر باشد، عملیات خرید انجام نمی‌شود
            if net_buy_amount > balance:
                print("Not enough balance to buy the shares!")
            else:
                # فروش سهام قبلی
                shares_sold = shares
                balance += sell_amount

                # خرید سهام جدید
                shares = shares_sold + buy_shares
                balance -= buy_amount

                # چاپ جزئیات عملیات
                print("Bought", buy_shares, "shares at price", data['Close'][i+1+split], "for a total of", buy_amount)
                print("Sold", shares_sold, "shares at price", data['Close'][i+1+split], "for a total of", sell_amount)
                print("Net buy amount:", net_buy_amount)
                print("New balance:", balance)
                print("Profit rate:", profit_rate)